In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import missingno as msno

In [3]:
# import pickle
df_sugarbeet = pd.read_pickle('data/sugarbeet.pkl')
df_weather_monthly = pd.read_pickle('data/df_weatherstations_monthly.pkl')

In [4]:
df_sugarbeet.station_location.unique()

array(['Emmeloord', 'Lelystad', 'Rittershausen', 'Sommepy', 'Herchsheim',
       'Lamotte', 'Mattenkofen', 'Pithiviers', 'Vierhoefen', 'Bautzen',
       'Stadthagen', 'Goderville', 'Soest', 'Anklam'], dtype=object)

In [3]:
# reset index of weather frame
df_weather_monthly = df_weather_monthly.reset_index()
# drop index column created by step above
df_weather_monthly.drop(['index'], axis=1, inplace=True)

In [4]:
# create lists of columns to include in different pivot tables
# long list for more complicated/detailed analysis, short list for baseline model
pivotvaluelist = ['air_temperature_avg_monthly',
       'air_temperature_max_monthly', 'air_temperature_min_monthly',
       'eag_soil_moisture_1_monthly', 'eag_soil_moisture_2_monthly',
       'eag_soil_moisture_3_monthly', 'eag_soil_moisture_4_monthly',
       'eag_soil_moisture_5_monthly', 'eag_soil_moisture_6_monthly',
       'soil_salinity_1_monthly', 'soil_salinity_2_monthly',
       'soil_salinity_3_monthly', 'soil_salinity_4_monthly',
       'soil_salinity_5_monthly', 'soil_salinity_6_monthly',
       'soil_temperature_1_vg_monthly', 'soil_temperature_2_vg_monthly',
       'soil_temperature_3_vg_monthly', 'soil_temperature_4_vg_monthly',
       'soil_temperature_5_vg_monthly', 'soil_temperature_6_vg_monthly',
       'soil_temperature_1_min_monthly', 'soil_temperature_2_min_monthly',
       'soil_temperature_3_min_monthly', 'soil_temperature_4_min_monthly',
       'soil_temperature_5_min_monthly', 'soil_temperature_6_min_monthly',
       'soil_temperature_1_max_monthly', 'soil_temperature_2_max_monthly',
       'soil_temperature_3_max_monthly', 'soil_temperature_4_max_monthly',
       'soil_temperature_5_max_monthly', 'soil_temperature_6_max_monthly',
       'solar_radiation_monthly', 'relative_humidity_avg_monthly',
       'relative_humidity_max_monthly', 'relative_humidity_min_monthly',
       'precipitation_monthly', 'leaf_wetness_monthly']
simplepivotvaluelist = ['air_temperature_avg_monthly', 'air_temperature_max_monthly', 'air_temperature_min_monthly']

In [5]:
# make column name to str to facilitate flattening in the pivot dataframe later
df_weather_monthly.month = df_weather_monthly.month.astype(str)

In [6]:
# change dataframe shape from long to wide with pivot
df_weather_piv = pd.pivot(df_weather_monthly, index='station_location', columns=['month'], values=simplepivotvaluelist)

In [7]:
# flatten the multi-index columns
df_weather_piv.columns = ['_'.join(col) for col in df_weather_piv.columns.values]
#df_weather_piv.columns = ["_".join(tuple(map(str, t)))for t in df_weather_piv.columns.values]
# flatten all columns to one level
df_weather_piv.reset_index()

,station_location,air_temperature_avg_monthly_10,air_temperature_avg_monthly_11,air_temperature_avg_monthly_3,air_temperature_avg_monthly_4,air_temperature_avg_monthly_5,air_temperature_avg_monthly_6,air_temperature_avg_monthly_7,air_temperature_avg_monthly_8,air_temperature_avg_monthly_9,...,air_temperature_max_monthly_9,air_temperature_min_monthly_10,air_temperature_min_monthly_11,air_temperature_min_monthly_3,air_temperature_min_monthly_4,air_temperature_min_monthly_5,air_temperature_min_monthly_6,air_temperature_min_monthly_7,air_temperature_min_monthly_8,air_temperature_min_monthly_9
0,Anklam,10.007846,NaN,NaN,6.613427,11.336505,19.068611,19.058535,15.815605,14.945822,...,15.446769,9.585241,NaN,NaN,6.057263,10.793441,18.454750,18.436196,15.209852,14.468329
1,Bautzen,NaN,NaN,NaN,11.117188,11.611196,16.587822,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.467187,11.034355,15.829406,NaN,NaN,NaN
2,Berklingen,NaN,NaN,NaN,5.770699,11.239247,18.984361,18.554449,16.557513,13.525556,...,13.765556,NaN,NaN,NaN,5.182568,10.644610,18.392597,17.970605,15.950027,13.306667
3,Emmeloord,NaN,NaN,NaN,7.942705,11.173804,17.907889,17.906169,16.754032,15.992228,...,16.535924,NaN,NaN,NaN,7.477295,10.757823,17.443014,17.403562,16.251855,15.461739
4,Groningen,NaN,NaN,NaN,7.490939,10.235892,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.990884,9.812649,NaN,NaN,NaN,NaN
5,Herchsheim 1,10.005588,NaN,NaN,8.044080,11.056371,19.320847,17.790497,16.541895,15.200209,...,15.914576,9.281176,NaN,NaN,7.453520,10.447796,18.705764,17.120618,15.877473,14.504103
6,Lamotte,11.142928,NaN,NaN,9.624444,11.654073,17.851292,18.367688,17.027594,16.756444,...,17.359219,10.635168,NaN,NaN,8.987778,10.985780,17.283694,17.771478,16.411183,16.168773
7,Lelystad,NaN,NaN,NaN,NaN,NaN,17.817029,17.844368,16.813495,16.812764,...,17.364327,NaN,NaN,NaN,NaN,NaN,17.361868,17.362796,16.319583,16.256364
8,Mattenkofen,NaN,NaN,12.723636,7.009264,11.391723,19.779389,18.421949,16.712661,14.927701,...,15.654861,NaN,NaN,11.751212,6.350958,10.854576,19.080792,17.764489,16.079005,14.223997
9,Peine,11.048023,NaN,NaN,7.849278,11.726720,19.580667,18.564933,16.745148,15.152253,...,15.674117,10.507450,NaN,NaN,7.227056,11.184261,19.007208,17.975417,16.159005,14.652573


In [8]:
# merge sugar beet dataframe with the monthly weather info
df_merge_monthly = df_sugarbeet.merge(df_weather_piv, 
                                      on='station_location',
                                      how='outer')

In [9]:
# compare shapes of original and new merge dataframe
# --> we gained some rows
print(f'the sugarbeet dataframe has {df_sugarbeet.shape[0]} rows and {df_sugarbeet.shape[1]} columns')
print(f'the sugarbeet dataframe has {df_merge_monthly.shape[0]} rows and {df_merge_monthly.shape[1]} columns')

the sugarbeet dataframe has 16477 rows and 19 columns
the sugarbeet dataframe has 16482 rows and 46 columns


In [10]:
df_merge_monthly.groupby(['station_location']).mean()

/var/folders/r8/96vp6rrx3633scm80v59y52c0000gn/T/ipykernel_2960/4228447699.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_merge_monthly.groupby(['station_location']).mean()


,betaine_nir,cry_nir,csy_nir,dm_nir,invert_nir,mark_nir,ms_comp,obj,otype_comp,pollinator_comp,...,air_temperature_max_monthly_9,air_temperature_min_monthly_10,air_temperature_min_monthly_11,air_temperature_min_monthly_3,air_temperature_min_monthly_4,air_temperature_min_monthly_5,air_temperature_min_monthly_6,air_temperature_min_monthly_7,air_temperature_min_monthly_8,air_temperature_min_monthly_9
station_location,,,,,,,,,,,,,,,,,,,,,
Anklam,0.118164,60.410328,10.247223,21.789179,0.081215,4.815650,1.750357,30.583452,3.119829,567.205421,...,15.446769,9.585241,NaN,NaN,6.057263,10.793441,18.454750,18.436196,15.209852,14.468329
Bautzen,0.242435,68.591283,11.361550,22.157877,0.240900,5.584027,1.545833,34.495000,3.335000,682.448333,...,NaN,NaN,NaN,NaN,10.467187,11.034355,15.829406,NaN,NaN,NaN
Berklingen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.765556,NaN,NaN,NaN,5.182568,10.644610,18.392597,17.970605,15.950027,13.306667
Emmeloord,0.120584,53.409332,8.488441,21.425727,0.171103,5.529328,1.550543,34.309942,3.335840,680.071011,...,16.535924,NaN,NaN,NaN,7.477295,10.757823,17.443014,17.403562,16.251855,15.461739
Goderville,0.127166,61.337006,10.234917,22.277686,0.098934,5.639084,1.595777,34.677225,3.277526,653.582202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Groningen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6.990884,9.812649,NaN,NaN,NaN,NaN
Herchsheim,0.169137,67.267432,12.460393,23.914446,0.123959,5.409333,1.972973,28.797297,2.932432,345.770270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Herchsheim 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.914576,9.281176,NaN,NaN,7.453520,10.447796,18.705764,17.120618,15.877473,14.504103
Lamotte,0.144019,74.656006,12.798154,23.559348,0.105824,6.406146,1.586317,33.578361,3.280032,663.757359,...,17.359219,10.635168,NaN,NaN,8.987778,10.985780,17.283694,17.771478,16.411183,16.168773


In [11]:
df_merge_monthly.isnull().sum()

betaine_nir                           5
cry_nir                               5
csy_nir                               5
dm_nir                                5
fieldid                               5
region                                5
invert_nir                            5
mark_nir                              5
ms_comp                               5
obj                                   5
otype_comp                            5
pollinator_comp                       5
sc_nir                                5
seednames_coded                       5
seriesid                              5
totaln_nir                            5
x                                     5
y                                     5
station_location                      0
air_temperature_avg_monthly_10    11293
air_temperature_avg_monthly_11    15125
air_temperature_avg_monthly_3     15075
air_temperature_avg_monthly_4      6601
air_temperature_avg_monthly_5      6601
air_temperature_avg_monthly_6      5641


In [12]:
df_merge_monthly.air_temperature_avg_monthly_6

0        17.907889
1        17.907889
2        17.907889
3        17.907889
4        17.907889
           ...    
16477    18.984361
16478          NaN
16479    19.320847
16480    19.580667
16481    19.285042
Name: air_temperature_avg_monthly_6, Length: 16482, dtype: float64

In [13]:
locationdroplist = ['Berklingen', 'Groningen', 'Herchsheim 1', 'Peine', 'Söllingen', 'Bautzen', 'Lamotte', 'Lelystad', 'Rittershausen']

In [14]:
# create a baseline model df with the core dates where we have weather data,
df_baseline_model = df_merge_monthly.drop(['air_temperature_avg_monthly_10', 'air_temperature_avg_monthly_11', 'air_temperature_avg_monthly_3',
                                           'air_temperature_max_monthly_10', 'air_temperature_max_monthly_11', 'air_temperature_max_monthly_3',
                                           'air_temperature_min_monthly_10', 'air_temperature_min_monthly_11', 'air_temperature_min_monthly_3'],
                                           axis=1)


In [15]:
# drop location with lots of missing values
df_baseline_model.drop(df_baseline_model[
    df_baseline_model.station_location
    .isin(locationdroplist)]
    .index, 
    axis=0, 
    inplace=True)
#df_baseline_model.drop(df_baseline_model.loc[df_baseline_model['station_location'].isin(locationdroplist)], inplace=True)

In [16]:
df_baseline_model.shape

(11895, 37)

In [17]:
df_baseline_model.dropna(inplace=True)
df_baseline_model.station_location.unique()

array(['Emmeloord', 'Sommepy', 'Mattenkofen', 'Stadthagen', 'Soest',
       'Anklam'], dtype=object)

In [18]:
df_baseline_model.shape

(7419, 37)

In [19]:
df_baseline_model.to_pickle('data/baseline_model.pkl')